<a href="https://colab.research.google.com/github/marcuslovinit/hello-world/blob/master/MNIST_nn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
import torch
from torch import nn
from torch import optim
from torchvision import datasets, transforms
from torch.utils.data import random_split, DataLoader

In [29]:
torch.randn(5).cuda()

tensor([-3.7314, -0.9945, -1.2475, -0.9075, -1.0544], device='cuda:0')

In [6]:
model = nn.Sequential(
    nn.Linear(28 * 28, 64), #first hidden layer has 64 units
    nn.ReLU(),
    nn.Linear(64, 64),
    nn.ReLU(),
    nn.Linear(64, 10)
)

In [31]:
class ResNet(nn.Module):
  def _init_(self):
    super()._init_()
    self.l1 = nn.Linear(28 * 28, 64)
    self.l2 = nn.Linear(64, 64)
    self.l3 = nn.Linear(64, 10)
    self.do = nn.Dropout(0.1)
  
  def forward(self, x):
    h1 = nn.functional.relu(self.l1(x))
    h2 = nn.functional.relu(self.l2(h1))
    do = self.do(h2 + h1)
    logits = self.l3(do)
    return logits
  
model = ResNet().cuda()

In [32]:
# Define my optimiser, not necessary but convenient way to do nested update of all model parameters
params = model.parameters()
optimiser = optim.SGD(params, lr=1e-2) #learning rate at 100

ValueError: ignored

In [33]:
#define my loss -> defining an objective function that's going to update parameters
loss = nn.CrossEntropyLoss()

In [30]:
# Train, Validation split
train_data = datasets.MNIST('data', train=True, download=True, transform=transforms.ToTensor())
train, val = random_split(train_data, [55000, 5000])
train_loader = DataLoader(train, batch_size=32)
val_loader = DataLoader(val, batch_size=32)


In [39]:
#my training and validation loops
nb_epochs = 5 #no.of times i go through dataset
for epoch in range(nb_epochs):
  losses = list() #to keep track of value of loss during training
  accuracies = list() #to keep track of value of accuracy during training
  model.train() #because I use Dropout
  for batch in train_loader:
    x, y = batch
    # x: b x 1 x 28 x 28 where x is going to be an image batch size x 1 channel x 28 x 28
    b = x.size(0) #reshape x such taht it goes from a 28x28 image into just a vector which is 28 square
    x = x.view(b, -1).cuda() #matrix with b rows, length will be 28 squares

#following steps are always present in supervised learning.
    # step 1: forward
    with torch.no_grad():
      l = model(x) #l: logits (output of the last linear module in yr model)

    # step 2: compute the objective function
    J = loss(l, y.cuda()) #measure how well the model is performing, y is the label. distance between the network performance to the final task. high loss = network is far from achieving the task the model is asked to perform
    # objective loss = loss in volume but then 1 is functional output of the network, other is functional parameters of the network. 

#note steps 3, 4, 5 is specifically for training model
    # step 3: cleaning the gradients from previous steps
    model.zero_grad() #note that you can also zero gradient using optimiser cuz the model para & optimiser para are the same 
    #optimiser.zero_grad()
    #params.grad._zero()

    # step 4: accumulate the partial derivatives of J wrt the parameters/accumulates new grads with previous ones
    J.backward()
    #params.grad.add_(dJ/dparams) note: since we dw to keep all the summations, only want the latest grad to do stochatist. he has the cleaning grad step. it's only necessary when you want the partial instead of accumulation.
    #why pytorch accum instead compute? cuz u can do many more stuff.
    
    # step 5: step in the opposite direction of the gradient. change of parameters to minimise the objective functions.
    optimiser.step()
    #with torch.no_grad(): params = params - eta * params.grad

    losses.append(J.item()) # good to call item b4 putting it into an array. if u din do that, u will run out of memory quickly.

  print(f'Epoch {epoch + 1}, train loss: {torch.tensor(losses).mean():.2f}') #2f is to print 2d.p. (plotting latest J, the last J computed)

  losses = list()
  for batch in train_loader:
    x, y = batch
    # x: b x 1 x 28 x 28 where x is going to be an image batch size x 1 channel x 28 x 28
    b = x.size(0)
    x = x.view(b, -1).cuda() #matrix with b rows, length will be 28 squares

    # step 1: forward
    with torch.no_grad():
      l = model(x) 
      #compute logits without computational graphs
      #l: logits (output of the last linear module in yr model)

    # step 2: compute the objective function
    J = loss(l, y.cuda()) #measure how well the model is performing, y is the label. distance between the network performance to the final task. high loss = network is far from achieving the task the model is asked to perform
    # objective loss = loss in volume but then 1 is functional output of the network, other is functional parameters of the network. 
    
    losses.append(J.item()) #append item to the list
    accuracies.append(y.eq(l.detach().argmax(dim=1).cpu()).float().mean())

  print(f'Epoch {epoch + 1}, validation loss: {torch.tensor(losses).mean():.2f}') #2f is to print 2d.p.

ModuleAttributeError: ignored